In [1]:
%load_ext autoreload
%autoreload 2

In [16]:
# from ragas.executor import Executor
# from asyncio import sleep

# exec = Executor(raise_exceptions=True)
# for i in range(10):
#     exec.submit(sleep, i)

# try:
#     exec.results()
# except Exception:
#     print("error")

Evaluating:   0%|          | 0/10 [00:00<?, ?it/s]

In [ ]:
from ragas.executor import is_event_loop_running

assert is_event_loop_running() is True